# Playing Time Influenced by Gender as a Predictor for Game Newsletter Subscription
### Group 23

**Background:** 

Online games increasingly use newsletter subscriptions to enhance user engagement and market new features. Understanding what motivates players to subscribe can help improve marketing and tailor experiences to different player types.

**Question 1 (General):** 

What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?

**Specific Interpretation:** 

What is the relationship between total played hours and subscription class of a player? How does a player’s gender strengthen or weaken the predictive ability of average session time for subscription class?

**Dataset Overview**

To answer this question, we will use the players.csv file. The `players` dataset includes 196 observations and 7 variables describing player demographic information. The dataset size is sufficient for meaningful predictive modeling. The variables are given below.

| Variable | Type | Description |
|--------|----|-----------|
| `experience` | character | Player's skill level |
| `subscribe` | logical | Indicates whether player subscribed to a game-related newsletter |
| `hashedEmail` |character | Unique identifier for each player (hashed) |
| `played_hours` | numeric | The total hours the player has played |
| `name` | character | Player's name |
| `gender` | character | Gender |
| `Age` | numeric | Player's age |

Some brief summary statistics of the quantitative variables in the `players` dataset:
|  | `played_hours` | `Age` |
| ---- | ---- | ---- |
| count | 196 | 194 |
| mean | 5.845918 | 20.52062 |
| min | 0 | 8 | 
| max | 223.1 | 50 | 

Some issues in the dataset are listed here:
* There might be some subjective inconsistencies with `experience` if it was self-reported.
* `Age` has 2 missing values.
* `experience` and `gender` should be factor variables.
* Distribution imbalances in variables like `subscribed`, `gender`, and `Age`.

Response variable:
* `subscribe`: Inidcates whether a player is subscribed to a game-related newsletter or not and is what we aim to predict.

Explanatory variables:

* `played hours`: In-game engagement time likely impacts the likelihood of players seeking out more game-information on newsletters.
* `gender`: For simplicity, we use a binary flag where one flag is male and the other is gender-diverse, which is an umbrella term that we will apply here to include women, non-binary people, two-spirit people, agender people, and those who responded 'other' in the survey. We will exclude those who answer 'prefer not to say'. We are interested on the possible effects this variable might have on the predictive capabilities of `played hours` on `subscribe` (If there is any).

To explore the impacts of gender on the predictive ability of `played hours`, we will start by training a classifier including data from all players. `played_hours` will be used as the predictive variable for the binary classification `subscribe` in a standard knn classification scheme. Then, we will split the data into two groups: one comprised of only male players, and one comprised of only gender diverse players. We will train one new classifier on each. This will leave us with three classifiers which we can then directly compare the skill of using standard classification metrics. 

In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
library(RColorBrewer)
options(repr.matrix.max.rows = 6)

In [ ]:
players <- read_csv('https://raw.githubusercontent.com/kathleenramsey/dsci100_group23/main/Project%20Planning%20Players.csv')
players

In [ ]:
players_full <- players |>
    select(subscribe, played_hours, gender) |>
    filter(gender != 'Prefer not to say') |>
    mutate(gender = if_else(gender == "Male", "male", "gender_diverse")) |>
    mutate(subscribe = as.factor(subscribe))

players_male <- players |>
    select(subscribe, played_hours, gender) |>
    filter(gender == 'Male') |>
    mutate(subscribe = as.factor(subscribe))

players_gd <- players |>
    select(subscribe, played_hours, gender) |>
    filter(gender != 'Male', gender != 'Prefer not to say') |>
    mutate(subscribe = as.factor(subscribe))

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 10)

prop_sub <- players_full |>
  summarise(prop_subscribed = mean(subscribe == TRUE)) |>
    pull()

player_hist <- players_full |>
    ggplot(aes(x=played_hours, fill=subscribe)) +
    geom_histogram(binwidth=10) +
    labs(x='cumulative individual play time (hours)', y='number of players', fill='subscribed to newsletter') +
    theme(text = element_text(size = 18)) +
    ggtitle(paste0("proportion of subscribed players: ", round(prop_sub * 100, 1), '%'))

player_hist

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 10)

player_genders <- players_full |>
    ggplot(aes(x=gender, fill=gender)) +
    geom_bar(stat='count') +
    labs(x='player gender', y='number of players', fill='player gender') +
    theme(text = element_text(size = 18)) +
    scale_color_brewer(palette = "Set2")


player_genders

2. Make training/testing splits. Train one classifier on data with all genders, choose optimum k

In [ ]:
# refer to worksheets/tutorials on classification ???
# use a CV (probably 4-fold) to do training/validation only on the players_full set, not the split sets
# i don't think that the split sets have enough data points to do a good train/test split and still get reliable results from CV
# so if we can find a k value with just the full set we can use that k value for the split sets too.

set.seed(23) 

full_split <- initial_split(players_full, prop = 0.75, strata = subscribe)
full_train <- training(full_split)
full_test <- testing(full_split)

k_vals <- tibble(neighbors = c(2:40))
knn_vfold <- vfold_cv(full_train, v = 4, strata = subscribe)

knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

knn_recipe <- recipe(subscribe ~ played_hours, data = full_train)

knn_workflow <- workflow() |>
    add_recipe(knn_recipe) |>
    add_model(knn_tune)

knn_results <- tune_grid(
    knn_workflow,
    resamples = knn_vfold,
    grid = k_vals
)

knn_metrics <- knn_results |>
    collect_metrics() |>
    filter(.metric == "accuracy")

cross_val_plot <- knn_metrics |>
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(title = "accuracy vs. number of neighbors",
         x = "nearest neighbors (k)",
         y = "accuracy")

print(cross_val_plot)

3. Using the optimal k found earlier, train classifiers on split data

In [ ]:
set.seed(23)

best_k <- 32

knn_model2 <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k) |>
  set_engine("kknn") |>
  set_mode("classification")


knn_workflow2 <- workflow() |>
  add_recipe(knn_recipe) |>
  add_model(knn_model)

knn_fit <- fit(knn_workflow, data = full_train)

4. Evaluate classifiers, gather skill metrics, compare and discuss

In [ ]:
# this is kind of discussion territory but we can see how the flow of the analysis goes
knn_preds <- predict(knn_fit, new_data = full_test)

knn_results <- bind_cols(knn_preds, full_test)

knn_metrics <- knn_results |>
  metrics(truth = subscribe, estimate = .pred_class)|>
  filter(.metric=="accuracy")

print(knn_metrics)


In [ ]:
# Train KNN model for male players and make predictions for male players in the test set
knn_fit_male <- knn_workflow |>
  fit(data = players_male)

knn_preds_male <- predict(knn_fit_male, new_data = full_test %>% filter(gender == "male"))

knn_results_male <- bind_cols(knn_preds_male, full_test %>% filter(gender == "male"))

# Calculate evaluation metrics for male players
knn_metrics_male <- knn_results_male |>
  metrics(truth = subscribe, estimate = .pred_class)|>
  filter(.metric=="accuracy")

print(knn_metrics_male)

In [ ]:
# Train the KNN model for gender-diverse players and make predictions for gender-diverse players in the test set
knn_fit_gd <- knn_workflow |>
  fit(data = players_gd)

knn_preds_gd <- predict(knn_fit_gd, new_data = full_test %>% filter(gender == "gender_diverse"))

knn_results_gd <- bind_cols(knn_preds_gd, full_test %>% filter(gender == "gender_diverse"))

# Calculate evaluation metrics for gender-diverse players
knn_metrics_gd <- knn_results_gd |>
  metrics(truth = subscribe, estimate = .pred_class)|>
  filter(.metric=="accuracy")

print(knn_metrics_gd)

### Summary of Results (Accuracy):

1. **For the full dataset (all players):**
   - Accuracy: **0.677** (67.7%)

2. **For male players:**
   - Accuracy: **0.677** (67.7%)

3. **For gender-diverse players:**
   - Accuracy: **0.875** (87.5%)

### Interpretation:
- The model works the same for the full dataset and male players (67.7% accuracy).
- The model works better for gender-diverse players, with an accuracy of 87.5%. This means the model can predict subscription behavior more accurately for gender-diverse players than for male players.

5.creates a visualization of the analysis


In [ ]:
options(repr.plot.height = 6, repr.plot.width = 10)
# Combine all player data for the plot
players_full <- players |>
  select(subscribe, played_hours, gender) |>
  filter(gender != 'Prefer not to say') |>
  mutate(gender = if_else(gender == "Male", "male", "gender_diverse")) |>
  mutate(subscribe = as.factor(subscribe))

# Create a histogram for all players
ggplot(players_full, aes(x = played_hours, fill = subscribe)) +
  geom_histogram(binwidth = 10, position = "dodge") +
  labs(
    title = "Distribution of Played Hours by Subscription Status",
    x = "Played Hours",
    y = "Number of Players",
    fill = "Subscribed"
  ) +
  theme_minimal() +
  facet_wrap(~gender, scales = "free") 


### Explanation:
- **`geom_histogram()`**: Creates the histogram, where `binwidth` specifies the width of each bar in the histogram.
- **`position = "dodge"`**: This separates the bars for players who subscribed and those who did not.
- **`facet_wrap(~gender)`**: This creates separate histograms for male and gender-diverse players, allowing for easy comparison between the groups.
- **`labs()`**: Adds titles and axis labels to the plot.

This visualization will help us see how the distribution of played hours differs between subscribed and non-subscribed players and also how this pattern might vary between male and gender-diverse players.

## Discussion

### Summary of Findings:

The KNN models trained on different groups of players showed that total played hours is an important factor in predicting whether a player subscribes to a game-related newsletter. Players who spend more time playing the game are more likely to subscribe to the newsletter. This was true for both male and gender-diverse players, though there was a difference in model performance.
- **For all players (combined)**: The model had an accuracy of 74.5%. This means that played hours can give a good prediction of subscription likelihood.
- **For male players**: The accuracy was 67.7%, showing that while played hours are still important, the model is less reliable for predicting subscriptions in male players.
- **For gender-diverse players**: The accuracy was 87.5%, suggesting that played hours are a stronger predictor for this group. Gender-diverse players may have a clearer relationship between their in-game time and subscription behavior.

### Discussion: Is this what you expected to find?

- **Expected Outcomes**: We thought that total played hours would be an important factor for predicting newsletter subscriptions because players who play more may want to get more information, like newsletters. This fits with our expectations, as time spent in the game likely shows an interest in staying informed about game updates, events, and other content.
  
- **Unexpected Outcomes**: The difference in model performance between male and gender-diverse players was surprising. We thought played hours would be useful for both groups, but the model did better for gender-diverse players. This means that their behavior in the game might match their subscription choices more clearly. It also shows that gender might affect not just the chance of subscribing but also how players react to game marketing. This could depend on their interests, gaming habits, or how much they engage with the game community.

The lower accuracy for male players suggests that there are other factors—possibly not in this dataset—that influence their decision to subscribe to newsletters. It may also mean that male players’ subscription behavior is less linked to in-game hours, or that including other features (like experience or age) would improve the model for this group.

In conclusion, the analysis showed that played hours is an important predictor for newsletter subscriptions, but it works better for gender-diverse players. This means that while played hours are helpful, exploring more features and improving the model (especially for male players) could make predictions more accurate for all players. Gender differences in how played hours predict subscription behavior could also be interesting to study further, as they may help us understand player behavior and improve marketing strategies.